First create the required ophyd devices (eventually these will be in a separate, shared location)

In [1]:
from ophyd import Device, EpicsMotor, EpicsSignal, EpicsSignalRO
from ophyd import Component as Cpt

class DCM(Device):
    #upstream_x: EpicsMotor = Cpt(EpicsMotor, '-MO-DET-01:UPSTREAMX')

    bragg: EpicsMotor = Cpt(EpicsMotor, '-MO-DCM-01:BRAGG')
    roll: EpicsMotor = Cpt(EpicsMotor, '-MO-DCM-01:ROLL')
    offset: EpicsMotor = Cpt(EpicsMotor, '-MO-DCM-01:OFFSET')
    perp: EpicsMotor = Cpt(EpicsMotor, '-MO-DCM-01:PERP')
    energy: EpicsMotor = Cpt(EpicsMotor, '-MO-DCM-01:ENERGY')
    pitch: EpicsMotor = Cpt(EpicsMotor, '-MO-DCM-01:PITCH')
    wavelength: EpicsMotor = Cpt(EpicsMotor, '-MO-DCM-01:WAVELENGTH')

    #temperatures
    xtal1_temp: EpicsSignalRO = Cpt(EpicsSignalRO, '-MO-DCM-01:TEMP1')
    xtal2_temp: EpicsSignalRO = Cpt(EpicsSignalRO, '-MO-DCM-01:TEMP2')
    xtal1_heater_temp: EpicsSignalRO = Cpt(EpicsSignalRO, '-MO-DCM-01:TEMP3')
    xtal2_heater_temp: EpicsSignalRO = Cpt(EpicsSignalRO, '-MO-DCM-01:TEMP4')
    backplate_temp: EpicsSignalRO = Cpt(EpicsSignalRO, '-MO-DCM-01:TEMP5')
    perp_temp: EpicsSignalRO = Cpt(EpicsSignalRO, '-MO-DCM-01:TEMP6')
    perp_sub_assembly_temp: EpicsSignalRO = Cpt(EpicsSignalRO, '-MO-DCM-01:TEMP7')

In [2]:
from ophyd import Device, EpicsMotor, EpicsSignal, EpicsSignalRO
from ophyd import Component as Cpt
from ophyd import Kind

class QBPM1(Device):
    intensity: EpicsSignalRO = Cpt(EpicsSignalRO, '-DI-QBPM-01:INTEN', kind=Kind.normal)

Now actually run the scan

Some initial setup for running matplotlib

In [3]:
%matplotlib inline
from bluesky.utils import install_nb_kicker
install_nb_kicker()

In [ ]:
from bluesky import RunEngine
from devices.QBPM1 import QBPM1
from devices.DCM import DCM
from bluesky.callbacks import LiveTable
from bluesky.callbacks.fitting import PeakStats

import bluesky.plans as bp
import bluesky.plan_stubs as bps


qbpm1 = QBPM1(name="QBPM1", prefix="BL03I")
dcm = DCM(name="DCM", prefix="BL03I")

qbpm1.wait_for_connection()
dcm.wait_for_connection()

RE=RunEngine({})

ps = PeakStats('DCM_pitch', 'QBPM1_intensity')

RE(bp.rel_scan([qbpm1.intensity], dcm.pitch, -0.075, 0.075, 30),ps) 
LiveTable([dcm.pitch,qbpm1.intensity]))

print("centre",ps['com'])

RE(bps.mv(dcm.pitch, ps['com']))